In [9]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [10]:
#Testing
scrape_link = 'https://www.apartments.com/houses/ca/'

response = requests.get(scrape_link)
soup = BeautifulSoup(response.text, 'html.parser')
result = soup.find_all('div', class_="placardContainer")
result

[<div class="placardContainer" id="placardContainer">
 <ul>
 <li>
 <article class="prosumer placard" data-listingid="5mg949m" data-url="https://www.apartments.com/12093-hermon-dr-tustin-ca/5mg949m/">
 <section class="placardContent">
 <div class="media">
 <div class="imageContainer carousel slide" data-interval="false">
 <div class="carouselInner">
 <div class="item active " style='background-image: url("https://images1.apartments.com/i2/KTI5Q0lmPY5hTIVqP4R3YaeF6Keab8xt9tkNht-eZ-A/115/12093-hermon-dr-tustin-ca-building-photo.jpg");' title="Building Photo - 12093 Hermon Dr">
 </div>
 </div>
 </div>
 <span class="listingFreshness">
 <span class="lastUpdated">
 <i class="clockIcon" title="Last Updated"></i>
 <span>2 wks</span>
 </span>
 </span>
 </div>
 <div class="propertyInfo">
 <div class="infoPadding">
 <a class="placardTitle js-placardTitle " href="https://www.apartments.com/12093-hermon-dr-tustin-ca/5mg949m/" title="12093 Hermon Dr, Tustin, CA">
 12093 Hermon Dr</a>
 <a class="favor

In [11]:
#Set up list to hold response info
link_dict = []
all_property = []

In [13]:
# Loop through x pages of the website with filter of Single House
for scrape_links in [scrape_link.format(page) for page in range(1,2)]:
    res = requests.get(scrape_links)
    new_soup = BeautifulSoup(res.text, 'html.parser')
    new_results = new_soup.find_all('article', class_="placard")
    
    # Loop through returned results
    for result in new_results:
        find_tag = result.find('a')['href']
        link_dict.append(find_tag)

for property_link in link_dict:
    res = requests.get(property_link)
    new_soup = BeautifulSoup(res.text, 'html.parser')
    new_results = new_soup.find_all('div', class_="mainWrapper")
    
    
    for result in new_results:
        try:
            # Identify and return property name of Rental Place
            property_name = result.find('h1', class_="propertyName")
            # Identify and return address information of Rental Place
            clean_property_name = property_name.text.strip().replace("\n", "")
            street_data = result.find('div', class_="propertyAddress")
            street = street_data.text.strip().replace("\n", "").replace("\r", "")
            street_address = street.split(",")
            city = " ".join(street_address[0].split())
            state = " ".join(street_address[1].split())[:2]
            zip_code = " ".join(street_address[1].split())[2:7]
            # Identify and return number of bedroom and price of Rental Place
            bedroom_container = result.find('div', class_="rentRollupContainer")
            all_the_rents = bedroom_container.findAll('span', class_="rentRollup")
            
            bedroom_info = []
            for rent_block in all_the_rents:
                
                bedroom_type = rent_block.find('span', class_="longText").text
                try:
                    print(rent_block.text)
                    price = re.search('(?<!\.|\d)(\d+)', rent_block.text)
                    print(price)
                    price = price.group(3)
                except:
                    price = ""
                
                bedroom_info.append({
                    'bedroom_type': bedroom_type,
                    'price': price
                })
            
            rent_obj = {
                "Property_Name_Rent": clean_property_name,
                "City_Rent": city,
                "Type": "House",
                "State_Rent": state,
                "Zip_code": zip_code,
                "Bedroom_Info": bedroom_info,
                
            }
            
            for item in bedroom_info:
                rent_obj[item["bedroom_type"]] = item["price"]
                

        
            all_property.append(rent_obj)
        except AttributeError as e:
            print(e)

print(all_property)


4 Bedrooms
4 Beds

                        $4,900
                    
<_sre.SRE_Match object; span=(1, 2), match='4'>

4 Bedrooms
4 Beds

                        $1,950
                    
<_sre.SRE_Match object; span=(1, 2), match='4'>

3 Bedrooms
3 Beds

                        $3,000
                    
<_sre.SRE_Match object; span=(1, 2), match='3'>

4+ Bedrooms
4+ Beds

                        $5,200
                    
<_sre.SRE_Match object; span=(1, 2), match='4'>

4+ Bedrooms
4+ Beds

                        $5,799
                    
<_sre.SRE_Match object; span=(1, 2), match='4'>

4 Bedrooms
4 Beds

                        $4,250
                    
<_sre.SRE_Match object; span=(1, 2), match='4'>

4 Bedrooms
4 Beds

                        $4,000
                    
<_sre.SRE_Match object; span=(1, 2), match='4'>

4 Bedrooms
4 Beds

                        $5,150
                    
<_sre.SRE_Match object; span=(1, 2), match='4'>

4+ Bedrooms
4+ Beds

              

In [14]:
house_data = pd.DataFrame(all_property)
house_data

,2 Bedrooms,3 Bedrooms,4 Bedrooms,4+ Bedrooms,Bedroom_Info,City_Rent,Property_Name_Rent,State_Rent,Type,Zip_code
0,NaN,NaN,,NaN,"[{'bedroom_type': '4 Bedrooms', 'price': ''}]",Tustin,12093 Hermon Dr,CA,House,92782
1,NaN,NaN,,NaN,"[{'bedroom_type': '4 Bedrooms', 'price': ''}]",Frazier Park,2405 Yellowstone Ct,CA,House,93225
2,NaN,,NaN,NaN,"[{'bedroom_type': '3 Bedrooms', 'price': ''}]",Los Angeles,8527 Noble Ave,CA,House,91343
3,NaN,NaN,NaN,,"[{'bedroom_type': '4+ Bedrooms', 'price': ''}]",Yorba Linda,26082 Big Horn Mountain Way,CA,House,92887
4,NaN,NaN,NaN,,"[{'bedroom_type': '4+ Bedrooms', 'price': ''}]",Canyon Lake,29642 Vacation Dr,CA,House,92587
5,NaN,NaN,,NaN,"[{'bedroom_type': '4 Bedrooms', 'price': ''}]",Yorba Linda,24402 Avenida de Marcia,CA,House,92887
6,NaN,NaN,,NaN,"[{'bedroom_type': '4 Bedrooms', 'price': ''}]",Anaheim,437 S Westridge Cir,CA,House,92807
7,NaN,NaN,,NaN,"[{'bedroom_type': '4 Bedrooms', 'price': ''}]",Tustin,10887 Osterman Ave,CA,House,92782
8,NaN,,NaN,NaN,"[{'bedroom_type': '3 Bedrooms', 'price': ''}]",Chino Hills,16462 Turnbury Ct,CA,House,91709
9,NaN,NaN,NaN,,"[{'bedroom_type': '4+ Bedrooms', 'price': ''}]",Palmdale,37212 Liana Ln,CA,House,93551


In [6]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Q2-2018_Apartment_House_Data.csv", index=False, header=True)